In [1]:
from keras.applications.resnet50 import ResNet50

resnet = ResNet50()
with open("resnet50.txt", "w") as fp:
    resnet.summary(print_fn=lambda x: fp.write(x + "\r\n"))

C:\Users\hiroa\Anaconda3\envs\py27\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
import numpy as np
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from skimage.transform import resize
from IPython import embed


NUM_CLASSES = 10
BATCH_SIZE = 30
NUM_EPOCHS = 15
use_data_aug = True

# img_arr is of shape (n, h, w, c)
def resize_image_arr(img_arr):
    x_resized_list = []
    for i in range(img_arr.shape[0]):
        img = img_arr[0]
        resized_img = resize(img, (224, 224))
        x_resized_list.append(resized_img)
    return np.stack(x_resized_list)


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train[0:2]
y_train = y_train[0:2]
x_test = x_test[0:2]
y_test = y_test[0:2]

# Resize image arrays
x_train = resize_image_arr(x_train)
x_test = resize_image_arr(x_test)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

# Normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


base_model = ResNet50(include_top=False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)
# and a logistic layer -- 10 classes for CIFAR10
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0, decay=0.0)

# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.summary()

if not use_data_aug:
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS,
              validation_data=(x_test, y_test),
              shuffle=False)
else:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=BATCH_SIZE),
                        epochs=NUM_EPOCHS,
                        validation_data=(x_test, y_test),
                        steps_per_epoch=int(np.ceil(x_train.shape[0] / float(BATCH_SIZE))),
                        workers=1)

C:\Users\hiroa\Anaconda3\envs\py27\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\hiroa\Anaconda3\envs\py27\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


94658560/94653016 [==============================] - 39s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
___________________________________________

Epoch 1/15
1/1 [==============================] - 10s 10s/step - loss: 0.3150 - acc: 0.9000 - val_loss: 0.4102 - val_acc: 0.9000
Epoch 2/15
1/1 [==============================] - 3s 3s/step - loss: 0.3354 - acc: 0.9000 - val_loss: 0.4075 - val_acc: 0.9000
Epoch 3/15
1/1 [==============================] - 3s 3s/step - loss: 0.3150 - acc: 0.9000 - val_loss: 0.4075 - val_acc: 0.9000
Epoch 4/15
1/1 [==============================] - 3s 3s/step - loss: 0.3219 - acc: 0.9000 - val_loss: 0.4086 - val_acc: 0.9000
Epoch 5/15
1/1 [==============================] - 3s 3s/step - loss: 0.3150 - acc: 0.9000 - val_loss: 0.4105 - val_acc: 0.9000
Epoch 6/15
1/1 [==============================] - 3s 3s/step - loss: 0.3245 - acc: 0.9000 - val_loss: 0.4087 - val_acc: 0.9000
Epoch 7/15
1/1 [==============================] - 3s 3s/step - loss: 0.3150 - acc: 0.9000 - val_loss: 0.4096 - val_acc: 0.9000
Epoch 8/15
1/1 [==============================] - 3s 3s/step - loss: 0.3245 - acc: 0.9000 - val_loss: 0.4086 

In [3]:
model.save('resnet50_model.hdf5')